# Conjugate Heat Transfer for Flow Over a Pebble

This case considers the laminar flow around a pebble with a volumetric heat source. Information about the Cardinal tutorial can be found [here](https://cardinal.cels.anl.gov/tutorials/cht5.html).

<img src="https://cardinal.cels.anl.gov/media/pebble_1.png" width="300" height="300" />

This tutorial provides the data that will be used to showcase some basic `pyvista` functionality. 

## Reading data from Exodus and Nek5000 files

The pebble data is gzipped in the `pebble_cht` directory. To decompress it:
```bash
gzip -d -k pebble_cht/*.gz 
```

First, we will read the fluid mesh and fluid and solid results files using the `pv.get_reader` method which uses the file extension to determine which reader to use. The Exodus files are read using [`ExodusIIReader`](https://docs.pyvista.org/api/readers/_autosummary/pyvista.exodusiireader#pyvista.ExodusIIReader) and the NekRS data is read using [`Nek5000Reader`](https://docs.pyvista.org/api/readers/_autosummary/pyvista.nek5000reader).

In [2]:
import pyvista as pv
from pathlib import Path

path_base = Path("pebble_cht")

# get_reader function uses the file extensions to determine which reader should be used
fluid_mesh_reader = pv.get_reader(path_base / "pebble.exo")
fluid_reader = pv.get_reader(path_base / "pebble.nek5000")
solid_reader = pv.get_reader(path_base / "solid_out.e")

print(fluid_mesh_reader)
print(fluid_reader)
print(solid_reader)


ExodusIIReader('pebble_cht/pebble.exo')
Nek5000Reader('pebble_cht/pebble.nek5000')
ExodusIIReader('pebble_cht/solid_out.e')


## Some features of the ExodusIIReader

Here, we explore some of the basic operations related to the ExodusIIReader including handling of blocks and sets. By default, the ExodusIIReader the nodal and element block result arrays, which are interpretted as point and cell data, respectively. The full list of blocks and sets are
* *Element Blocks*
* Face Blocks
* Edge Blocks
* Element Sets
* *Side Sets*
* Face Sets
* Edge Sets
* *Node Sets*
At high-level these are exposed through the [`ExodusIIBlockSet`](https://docs.pyvista.org/api/readers/_autosummary/pyvista.exodusiiblockset#pyvista.ExodusIIBlockSet) class. To explore this further, we will have a closer look at the fluid mesh.

In [3]:
print(fluid_mesh_reader.element_blocks.names)
print(fluid_mesh_reader.side_sets.names)

['FLUID']
['INLET', 'OUTLET', 'INTERFACEL', 'WALL']


### MultiBlock datasets

The `ExodusIIReader`returns a `MultiBlock` dataset with a block for each block/set. These blocks are themselves `MultiBlock` datasets that contain a number of `UnstructuredGrid`s. `MultiBlock` objects are traversed like Python `Dict`s and can be indexed using the block name or an integer.

In [8]:
#Enable sidesets and read
fluid_mesh_reader.side_sets.enable_all()
fluid_mesh = fluid_mesh_reader.read()

# ExodusIIReader returns a MultiBlock dataset
print(fluid_mesh)

# Each block corresponds to the sets and blocks
# available in the Exodus II specification
for i, key in enumerate(fluid_mesh.keys(), 1):
    print(i, key)


MultiBlock (0x77edcc37e680)
  N Blocks:   8
  X Bounds:   -2.530e-02, 2.530e-02
  Y Bounds:   -2.530e-02, 2.530e-02
  Z Bounds:   -2.153e-01, 2.153e-01
1 Element Blocks
2 Face Blocks
3 Edge Blocks
4 Element Sets
5 Side Sets
6 Face Sets
7 Edge Sets
8 Node Sets


Let's look at the contents of the Side Sets MultiBlock Dataset

In [9]:

boundaries = fluid_mesh["Side Sets"].keys()
for boundary in boundaries:
    print("\n", boundary)
    print(fluid_mesh["Side Sets"][boundary], "\n")


 INLET
UnstructuredGrid (0x77edcc37e1a0)
  N Cells:    36
  N Points:   133
  X Bounds:   -2.530e-02, 2.530e-02
  Y Bounds:   -2.530e-02, 2.530e-02
  Z Bounds:   -2.153e-01, -2.153e-01
  N Arrays:   2 


 OUTLET
UnstructuredGrid (0x77edcc37e1a0)
  N Cells:    36
  N Points:   133
  X Bounds:   -2.530e-02, 2.530e-02
  Y Bounds:   -2.530e-02, 2.530e-02
  Z Bounds:   2.153e-01, 2.153e-01
  N Arrays:   2 


 INTERFACEL
UnstructuredGrid (0x77edcc37e1a0)
  N Cells:    96
  N Points:   290
  X Bounds:   -1.500e-02, 1.500e-02
  Y Bounds:   -1.500e-02, 1.500e-02
  Z Bounds:   -1.500e-02, 1.500e-02
  N Arrays:   2 


 WALL
UnstructuredGrid (0x77edcc37e1a0)
  N Cells:    1056
  N Points:   3216
  X Bounds:   -2.530e-02, 2.530e-02
  Y Bounds:   -2.530e-02, 2.530e-02
  Z Bounds:   -2.153e-01, 2.153e-01
  N Arrays:   2 



## Filters

Once data is loaded, the next step is usually to process the data in some way. In Pyvista, like Paraview this is done using filters.
- These mirror those in Paraview
- Include slices, clips, contours etc.

In [11]:
# lets look at the interior of the fluid mesh by clipping it through the pebble
clipped_side_sets = fluid_mesh["Side Sets"].clip(origin=(0,0,0),
                                                 normal='y')

clipped_blocks = fluid_mesh["Element Blocks"].clip(origin=(0,0,0),
                                                   normal='y')

print(clipped_side_sets)
print(clipped_blocks)

MultiBlock (0x77ed9c7558a0)
  N Blocks:   4
  X Bounds:   -2.530e-02, 2.530e-02
  Y Bounds:   -2.530e-02, 5.170e-26
  Z Bounds:   -2.153e-01, 2.153e-01
MultiBlock (0x77ed9c755120)
  N Blocks:   1
  X Bounds:   -2.530e-02, 2.530e-02
  Y Bounds:   -2.530e-02, 3.825e-03
  Z Bounds:   -2.153e-01, 2.153e-01


## Plotting
### Simple plots
The most basic plots can be produced using the `plot` method on the datasets. Below, we plot the MultiBlock dataset containing the side sets. 

In [12]:
clipped_side_sets.plot(window_size=(250,600),
                       zoom=2.,
                       color='gray')


Widget(value='<iframe src="http://localhost:42241/index.html?ui=P_0x77ed9c7527a0_2&reconnect=auto" class="pyvi…

In [13]:
clipped_blocks.plot(window_size=(250,600),
                       zoom=2.,
                       color='gray')

Widget(value='<iframe src="http://localhost:42241/index.html?ui=P_0x77edcc2ddf00_3&reconnect=auto" class="pyvi…

### Re-creating image from Tutorial

In this section, we re-create the image from the introduction with an interactive plot with linked views. 
- Introduce `Plotter` class: used to create more complex images.

<img src="https://cardinal.cels.anl.gov/media/pebble_1.png" width="300" height="300" />

In [15]:

# Create two side-by-side render windows
p = pv.Plotter(window_size=(500,600), shape=(1,2), border=False)

#select first render window and plot the element blocks
p.subplot(0,0)
p.add_mesh(clipped_blocks,
           color='gray',
           show_edges=True,
           edge_color='k')
p.add_axes()

#select second render window and plot each side set with a different color

p.subplot(0,1)
boundaries = clipped_side_sets.keys()
colors = ['r', 'g', 'b', 'y']
for boundary, color in zip(boundaries, colors):
    p.add_mesh(clipped_side_sets[boundary],
               color=color,
               show_edges=True,
               edge_color='k')
p.add_axes()

#link views and adjust camera
p.link_views()
p.camera.zoom(2.0)
p.show()


Widget(value='<iframe src="http://localhost:42241/index.html?ui=P_0x77ec6de0b8e0_5&reconnect=auto" class="pyvi…

## Visualise fluid and solid results

Here, we visualise the solid and fluid temperature field at the last output time point. We are interested in the results at the last time step, so we must set the time point or value before reading.

In [16]:
# Visualise flow

## set the active time point (can also set the time value)
solid_reader.set_active_time_point(solid_reader.number_time_points-1)
fluid_reader.set_active_time_point(fluid_reader.number_time_points-1)

print(f"Solid time: {solid_reader.active_time_value}.")
print(f"Fluid time: {fluid_reader.active_time_value}.")

# Use Data from the element blocks
solid = solid_reader.read()
solid_blocks = solid['Element Blocks'][0]

# Nek5000Reader returns Unstructuredgrid
fluid = fluid_reader.read()

print(solid_blocks)
print(fluid)

Solid time: 635.0.
Fluid time: 635.0.
3D-Mesh found, spectral element of size = 4*4*4=64
UnstructuredGrid (0x77ec6dd85d80)
  N Cells:    3584
  N Points:   3817
  X Bounds:   -1.500e-02, 1.500e-02
  Y Bounds:   -1.500e-02, 1.500e-02
  Z Bounds:   -1.500e-02, 1.500e-02
  N Arrays:   7
UnstructuredGrid (0x77ec1d6ce1a0)
  N Cells:    51408
  N Points:   121856
  X Bounds:   -2.530e-02, 2.530e-02
  Y Bounds:   -2.530e-02, 2.530e-02
  Z Bounds:   -2.153e-01, 2.153e-01
  N Arrays:   4


In [17]:
# plot show meshes using the plotter class
solid_clip = solid_blocks.clip(normal='y', origin=(0,0,0))
fluid_clip = fluid.clip(normal='y', origin=(0,0,0))

p = pv.Plotter(window_size=(250,600))
p.add_mesh(solid_clip,
           scalars='temp',
           cmap='bwr',
           clim=(0,600),
           show_edges=True,
           show_scalar_bar=False)

p.add_mesh(fluid_clip,
           scalars='Temperature',
           cmap='bwr',
           clim=(0,600),
           scalar_bar_args={'width' : 0.9,
                            'position_x' : 0.05,
                            'fmt' : "%.3g"})
p.camera.zoom(2.)
p.show()

Widget(value='<iframe src="http://localhost:42241/index.html?ui=P_0x77ec6de0ad10_6&reconnect=auto" class="pyvi…